# PFRA 

### _Development Notebook_
--- 

Tests using scrapbook

In [1]:
import sys;sys.path.append('../')

In [2]:
from hecrasio.core import *
from hecrasio.qaqc import *
import scrapbook as sb
import pandas as pd
import json
import shutil
from collections import ChainMap

In [3]:
# Make copies of example_nb
example_nb = "SB-QAQC-FluvialTest.ipynb"
for i in range(10):
    shutil.copy(example_nb , os.path.join(os.getcwd(), "temp", "SB-QAQC-FluvialTest-{}.ipynb".format(i) ))

In [4]:
# Create a large list of notebooks
nbs = os.listdir(os.path.join(os.getcwd(), "temp"))

In [5]:
# Read many notebooks
book = sb.read_notebooks(os.path.join(os.getcwd(), "temp"))

In [6]:
# Create a unified results dictionary
results_dict = {}
for tup in list(book.items()):
    nb, results = tup
    result_dict = dict(ChainMap(*[list(json.loads(scrap).values())[0] for scrap in results.scraps]))
    results_dict[nb] = result_dict

In [7]:
# Pull results into a large review table
results_table = pd.DataFrame.from_dict(results_dict).T
results_table.head(3)

,1D Cores,1D Methodology,1D2D Flow Tolerance,1D2D MaxIter,1D2D Minimum Flow Tolerance,1D2D WS Tolerance,2D Boundary Condition Ramp Up Fraction,2D Boundary Condition Volume Check,2D Cores,2D Coriolis,...,Vol Accounting 2D Bdry,Vol Accounting Cell End,Vol Accounting Cell Start,Vol Accounting DS Outflow,Vol Accounting Error,Vol Accounting Error Percentage,Vol Accounting Groundwater,"Vol Accounting LS, Pump Div",Vol Accounting Lat Hydro,Vol Accounting US Inflow
SB-QAQC-FluvialTest-0,0,Finite Difference,0.1,0,1,0.01,[0.1000000015],[False],0,False,...,0,99553.8,84515.9,0,36.8438,0.00171806,0,0,0,0
SB-QAQC-FluvialTest-1,0,Finite Difference,0.1,0,1,0.01,[0.1000000015],[False],0,False,...,0,99553.8,84515.9,0,36.8438,0.00171806,0,0,0,0
SB-QAQC-FluvialTest-2,0,Finite Difference,0.1,0,1,0.01,[0.1000000015],[False],0,False,...,0,99553.8,84515.9,0,36.8438,0.00171806,0,0,0,0


In [8]:
# Create a summary table
df = pd.DataFrame(columns=['Unique_Values', 'Warnings', 'Offending_Nbs'])
df['Result_Attribute'] = pd.Index(list(results_table.columns))
df.set_index('Result_Attribute', drop=True, inplace=True)
df.head(5)

,Unique_Values,Warnings,Offending_Nbs
Result_Attribute,,,
1D Cores,NaN,NaN,NaN
1D Methodology,NaN,NaN,NaN
1D2D Flow Tolerance,NaN,NaN,NaN
1D2D MaxIter,NaN,NaN,NaN
1D2D Minimum Flow Tolerance,NaN,NaN,NaN


In [9]:
df_w_uniques = identify_unique_values(df, results_table)
df_w_uniques.head(5)

,Unique_Values,Warnings,Offending_Nbs
Result_Attribute,,,
1D Cores,[0],NaN,NaN
1D Methodology,[Finite Difference],NaN,NaN
1D2D Flow Tolerance,[0.1000000015],NaN,NaN
1D2D MaxIter,[0],NaN,NaN
1D2D Minimum Flow Tolerance,[1.0],NaN,NaN


In [10]:
# Example warnings

for i in df_w_uniques.index:
    values = df_w_uniques.loc[i][0]
    if i == '1D Cores':
        validate_by_threshold(df_w_uniques, i, values, 0, results_table)
    elif i == 'Vol Accounting Error':
        validate_by_threshold(df_w_uniques, i, values, 30, results_table)
    elif i == 'Vol Accounting Error Percentage':
        validate_by_threshold(df_w_uniques, i, values, 0.001, results_table)

df_w_uniques.head(5)

,Unique_Values,Warnings,Offending_Nbs
Result_Attribute,,,
1D Cores,[0],PASS,[]
1D Methodology,[Finite Difference],NaN,NaN
1D2D Flow Tolerance,[0.1000000015],NaN,NaN
1D2D MaxIter,[0],NaN,NaN
1D2D Minimum Flow Tolerance,[1.0],NaN,NaN


### Possible evaluations
- Check output interval is the desired amount
- Check if solution populated with anything other than 'Unstead Finished Successfully'
- Check flow, geometry, and plan filenames have the same base

In [ ]:
def validate_by_equivalency(pd_df, attr, value_list, threshold, results_table_df):
    pd_df.loc[attr]['Warnings'] = 'WARNING' if any([value > threshold for value in value_list]) else 'PASS'
    pd_df.loc[attr]['Offending_Nbs'] = [results_table_df.index[i] for i, value in enumerate(list(results_table_df[attr])) if value > threshold]

In [406]:
def validate_by_text(pd_df, attr, value_list, required_text, results_table_df):
    pd_df.loc[attr]['Warnings'] = 'WARNING' if any([value != required_text for value in value_list]) else 'PASS'
    pd_df.loc[attr]['Offending_Nbs'] = [results_table_df.index[i] for i, value in enumerate(list(results_table_df[attr])) if value != required_text]

In [380]:
# Basic logic to check equivalency
interval_step = 15
interval_units = 'MIN'
intervals = [interval[:-3] for interval in test_df.loc['Base Output Interval'][0]]
units = [interval[-3:] for interval in test_df.loc['Base Output Interval'][0]]
'WARNING' if any([interval != str(interval_step) for interval in intervals]) else 'PASS'
'WARNING' if any([unit != str(interval_units) for unit in units]) else 'PASS'

# This needs to include something to evaluate these together...

'PASS'

In [408]:
# Impliment validate by text solution
for i in test_df.index:
    values = test_df.loc[i][0]
    if i == 'Solution':
        validate_by_text(test_df, i, values, 'Unsteady Finished Successfully', results_table)

In [416]:
# A number of these evaluations need to be done on the results table

# Basic logic for checking flow, geometry, and plan names match
for i in results_table.index:
    ff = results_table.loc[i]['Flow Filename'].split('.')[0]
    gf = results_table.loc[i]['Geometry Filename'].split('.')[0]
    pf = results_table.loc[i]['Plan Filename'].split('.')[0]
    print('WARNING') if any([name != ff for name in [ff, gf, pf]]) else print('PASS')

PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS


In [ ]:
# Could develop results table and the summary evaluator...this ends up being twice the validation


# END